# Minecraft Server über Python steuern


## Voraussetzung

### Server:
- Minecraft Java Version 1.15.2


- Minecraft Server:
  - https://getbukkit.org/download/spigot
    - ich nutze hier "spigot 1.15.2" Tuesday, January 21 2020
    - dazu die Datei spigot-1.15.2.jar herunterladen
  
  - plugins
    - **Mapcoords**: https://dev.bukkit.org/projects/mapcoords
      - ermöglich euch Spieler zu lokalisieren inkl. Map und Koordinaten
    - **Multiverse-Core**: https://dev.bukkit.org/projects/multiverse-core
      - Ermöglicht das anlegen mehrerer Maps auf einem Server
      - Mehr Infos https://github.com/Multiverse/Multiverse-Core/wiki/Command-Reference
    - WarpPortals: https://dev.bukkit.org/projects/warp-portals
      - Portale zwischen den einzelnen Maps erstellen
      - Commands Liste https://dev.bukkit.org/projects/warp-portals/pages/commands
    - *WorldEdit*: https://dev.bukkit.org/projects/worldedit
      - Ermöglicht dem Spieler Elemente/Bauten zu kopieren/einzufügen
      - Anlegen von Snippets/Schematics
      - Leider war es mir bisher nicht möglich, diese Funktionen remote auszuführen
      - Anleitung zur Nutzung https://www.planetminecraft.com/blog/the-complete-worldedit-tutorial-with-personal-tips-amp-tricks-2756799/
      
      
  
Die oben genannten Dateien wie folgt in einem eigenen Serverpfad ablegen. z.B.:
- Windows: **c:\minecraftserver**
- Linux **~/___user___/minecraftserver**

    |-- Serverpfad
        |-- spigot-1.15.2.jar
        |
        |-- server.properties   wird beim ersten start angelegt und muss editiert werden. s.u.
        |-- eula.txt            wird beim ersten start angelegt und muss editiert werden. s.u.
        |-- start.bat           euer start Batchfile für Windows
        |-- shart.sh            oder euer start shell file für Linux
        |
        |-- plugins
            |-- Mapcoords.jar
            |-- Multiverse-Core-4.1.0.jar
            
- eine ausführliche Anleitung gibt es hier https://minecraft.gamepedia.com/Tutorials/Setting_up_a_server
            

### Python:
- Zum Steuern die Python Lib mcrcon
  - https://pypi.org/project/mcrcon/
      - installieren mit: pip install mcrcon
      
  

### server.properties
#### für rcon vorbereiten
##### setze ein password
- rcon.password=MySecretKey
- enable-rcon=true
- rcon.port=25575


#### mehr Infos zu server.properties
  - https://minecraft-de.gamepedia.com/Server.properties

## start.bat
start "My Server" /min java -Xms512M -Xmx1G -XX:+UseConcMarkSweepGC -jar spigot-1.15.2.jar

oder einfach

java -Xms512M -Xmx1G -XX:+UseConcMarkSweepGC -jar spigot-1.15.2.jar

# Nutzung ___mcron___

In [ ]:
from mcrcon import MCRcon
# https://help.serverblend.com/en/article/rust-rcon-server-commands-ohd7z8/


In [ ]:
from secrets import credential
import secrets

In [ ]:
# servername = "127.0.0.1"
# mysecret = "MySecretKey"
# mysecret = 25575

servername = secrets.sys_local.servername
mysecret = secrets.sys_local.mysecret
myport = secrets.sys_local.myport

# player_name = "PixelLaeufer"
player_name = "YourTargetPlayerName"

### erst mal ein Kommando ausführen
- bsp: /list

In [ ]:
command = "list"
mcr = MCRcon(host=servername, password=mysecret, port=myport)
mcr.connect()
resp = mcr.command(command)
print(resp)
mcr.disconnect()

- Befehle werden ohne "/" eingegeben

### Übergebe das Minecraft command mit hilfe einer Funktuon
- dies hier ist nur eine Beipspiel Funktion
  - für jedes Kommando wird eine Verbindung hergestellt und wieder geschlossen
  - dieses Vorgehen ist für die Demonstration im BarCamp ausreichend
  - um die Geschwindigkeit vieler Aufeinanderfolgender Commands zu steigern sollte das jedoch geändert werden
  

In [ ]:
def get_response_command(command, parameter=None):
    mcr = MCRcon(host=servername, password=mysecret, port = myport)
    mcr.connect()
    if parameter == None:
        resp = mcr.command(command)
    else:
        resp = mcr.command(command + " " + parameter)
    mcr.disconnect()
    return resp

In [ ]:
# # Spieler zum Operator machen. Achtung: Operator Level beachten!
# command = "op " + player_name


command = "list"
get_response_command(command)

In [ ]:
get_response_command("time set day")

### Das Ganze als Funktion

- Tageszeit einstellen
- Wetter umstellen

In [ ]:
def set_time_day():
    return get_response_command("time", "set day")
def set_weather_clear():
    return get_response_command("weather", "clear")

In [ ]:
set_time_day()

In [ ]:
set_weather_clear()

### Das gleiche in der Erweiterung auf den Aufenthaltsort eines bestimmten Spielers bezogen

In [ ]:
def set_time_day_at_player(playername = ""):
    get_response_command("execute at {} anchored eyes run time set day".format(player_name))
def set_weather_clear_at_player(playername = ""):
    get_response_command("execute at {} anchored eyes run weather clear".format(player_name))
    

In [ ]:
set_time_day_at_player(playername = player_name)

### Möchtest du deine Umgebung aufräumen?
- Alle Monster und Tiere Töten
  - Es ist nur ein Spiel, als vegetarier würde ich das sonst nicht machen

In [ ]:
def kill_all_monsters_and_animals():
    get_response_command("kill @e[type=!minecraft:player]")

def kill_all_monsters_and_animals_at_player(playername = ""):
    get_response_command("execute at {} anchored eyes run kill @e[type=!minecraft:player]".format(playername))

In [ ]:
kill_all_monsters_and_animals()
kill_all_monsters_and_animals_at_player(player_name)

## GameMode für den Spieler ändern

In [ ]:
gamemodes = ["creative", "creative", "adventure", "spectator"]

In [ ]:
# get_response_command("gamemode creative " + player_name)

In [ ]:
# get_response_command("gamemode survival " + player_name)

In [ ]:
def set_gamemode_for_player(player, gamemode):
    return get_response_command("gamemode " + gamemode + " " + player)

In [ ]:
set_gamemode_for_player(player_name, gamemodes[1])

## Nochmal zurück zur Liste der aktuellen Spieler

In [ ]:
command = "list"
get_response_command(command)

In [ ]:
def get_online_players_separated():
    resp = get_response_command("list")

    # print(resp)
    if "There are" in resp:
        counts = resp.split("There are ")[1]
        counts = int(counts.split(" of a max")[0])
        players = resp.split("players online: ")[1]
        players = players.split(", ")
        return [counts, players]
    else:
        return [-1, [''] ]

In [ ]:
def get_online_players():
    resp = get_response_command("list")
    # print(resp)
    if "There are" in resp:
        counts = resp.split("There are ")[1]
        counts = int(counts.split(" of a max")[0])
        players = resp.split("players online: ")[1].split(", ")
        return [counts, players]
    else:
        return [-1, [''] ]

In [ ]:
get_online_players()

### Nun einzelne Spieler lokaisieren mit dem Plugin mapcoords
- Befehl: "mc find"

In [ ]:
get_response_command("mc find " + player_name)

In [ ]:
def get_players_location(name):
    # return [PlayerName, Map, [x, y, z]]
    result = get_response_command("mc find " + str(name))
    if not "isn't online" in result:
        result = result.replace("§6", "")
        result = result.replace("§f", "")
        result = result.replace("§d", "")
        result = result.replace("§b", "")
        Player = result.split(" is at ")
        Map = Player[1].split(" X: ")[0].split("[")[1]
        Map = Map.split("]")[0]
        mystr = Player[1]
        newstr = ''.join((ch if ch in '0123456789.-' else ' ') for ch in mystr)
        numbers = [float(i) for i in newstr.split()]
        Coords = [int(numbers[-3]), int(numbers[-2]), int(numbers[-1])]
        return [Player[0], Map, Coords]
    else:
        return []


In [ ]:
get_players_location(player_name)

### Jetzt möchte ich eine Liste aller Spieler erhalten
- mit Welt und Koordinalten

#### Rückgabe als Liste

In [ ]:
def get_all_online_players_location():
    # return [ [PlayerName1, Map, [x, y, z]], [PlayerName2, Map, [x, y, z]] ]
    online_list = get_online_players()
    player_locations = []
    for index in range(online_list[0]):
        player_locations.append(get_players_location(online_list[1][index]))
    return player_locations

In [ ]:
get_all_online_players_location()

#### Rückgabe als Dict

In [ ]:
def get_player_locations_as_dict():
    # return {'PlayerName1': ['Overworld', [249, 68, 205]], 
    #         'PlayerName2': ['Overworld', [249, 68, 205]]}
    players_dict = {}
    result = get_all_online_players_location()
    for each in result:
        players_dict[each[0]] = [each[1], each[2]]
    return players_dict

In [ ]:
players_dict = get_player_locations_as_dict()
players_dict

### Blöcke an der einer beliebigen Position setzen
- Hier an der Position des Spielers

- xyz-Koordinaten müssen hier als str() übergeben werden
- coords = ["~", "~", "~"]

In [ ]:
def set_block_at_player_coords(blockname = "minecraft:stone", player = "", coords = ["~", "~", "~"]):
    coordinates = " ".join(coords)
    get_response_command("execute at {} anchored eyes run setblock {} {}".format(player, coordinates, blockname))
    

In [ ]:
block_name = "minecraft:stone"

set_block_at_player_coords(blockname = block_name, player = player_name, coords = ["~", "~", "~"])

- hier an eine festen Position
  - geht nur mit aktiven Chunks

In [ ]:
players = get_all_online_players_location()
print(players)
for player in players:
    coords = player[2]
    print(int(coords[0]), int(coords[1]), int(coords[2]))

- hier besorge ich zunächst die Spieler Position

In [ ]:
players_dict = get_player_locations_as_dict()
print(players_dict)
print(players_dict[player_name][1])

In [ ]:
orig_coords_backup = players_dict[player_name].copy()
print(orig_coords_backup)

In [ ]:
# coords = [252, 68, 212] # Koordinaten evtl merken

def coordstostring(coords):
    # input coords = [int(), int(), int()]
    # return [str(int()), str(int()), str(int())]
    coords=[str(int(coords[0])), str(int(coords[1])), str(int(coords[2]))]
    return coords


In [ ]:
string_coords=coordstostring(coords)

blockname = "minecraft:stone"
set_block_at_player_coords(blockname = blockname, player = player_name, coords = string_coords)

In [ ]:
string_coords

#### Generiere eine Platte an der stelle orig_coords

In [ ]:
orig_coords = orig_coords_backup[1].copy()
print(orig_coords)

blockname = "minecraft:stone"
# blockname = "minecraft:air"

radius = 6   # radius definiere
y_offset = 0 # höhen offset definieren



orig_coords[1] += y_offset
orig_coords[0] -= radius
orig_coords[2] -= radius

for x in range(radius*2 +1):
    coords = orig_coords.copy()
    coords[0] = orig_coords[0] + x
#     print(coords)
    string_coords=coordstostring(coords)
    set_block_at_player_coords(blockname = blockname, player = player, coords = string_coords)
    for y in range(radius*2):
        coords[2] = coords[2] + 1
#         print(coords)
#         print(x,y, string_coords)
        string_coords=coordstostring(coords)
        set_block_at_player_coords(blockname = blockname, player = player, coords = string_coords)
# print(x,y, string_coords)

In [ ]:
get_response_command("setblock 252 68 212 minecraft:stone")

### Generiere eine 4 Wände den an der stelle orig_coords
- mit einerm definierten Umfang

In [ ]:
orig_coords = orig_coords_backup[1].copy()
print(orig_coords)

# blockname = "minecraft:gold_block"
blockname = "minecraft:gold_ore"
# blockname = "minecraft:stone"
# blockname = "minecraft:lava"
# blockname = "minecraft:water"
# blockname = "minecraft:air"

player = player_name
radius = 6 # radius definieren
high = 5   # bauhöhe definieren

y_offset = 1

players_dict = get_player_locations_as_dict()

if player in players_dict:
#     orig_coords = players_dict[player][1].copy()
#     orig_coords_backup = orig_coords.copy()
    orig_coords = orig_coords_backup[1].copy()
    orig_coords[1] += y_offset
    print(orig_coords)
    # orig_coords=[103, 4, -123]

    orig_coords[0] -= radius
    orig_coords[2] -= radius

    print(orig_coords)
    effect = True
    for z in range(high):
        coords = orig_coords.copy()
        coords[1] += z
        for x in range(radius*2):
        #     actual = coords.copy()
            coords[0] += 1
#             print(coords)
            string_coords=coordstostring(coords)
            if effect: set_block_at_player_coords(blockname = blockname, player = player, coords = string_coords)
        print(x,y,z, string_coords)
        for y in range(radius*2):
        #     actual = coords.copy()
            coords[2] += 1
#             print(coords)
            string_coords=coordstostring(coords)
            if effect: set_block_at_player_coords(blockname = blockname, player = player, coords = string_coords)
        print(x,y,z, string_coords)
        for x in range(radius*2):
        #     actual = coords.copy()
            coords[0] -= 1
#             print(coords)
            string_coords=coordstostring(coords)
            if effect: set_block_at_player_coords(blockname = blockname, player = player, coords = string_coords)
        print(x,y,z, string_coords)
        for y in range(radius*2):
        #     actual = coords.copy()
            coords[2] -= 1
#             print(coords)
            string_coords=coordstostring(coords)
            if effect: set_block_at_player_coords(blockname = blockname, player = player, coords = string_coords)


## Infos zu Multiverse und WarpPortals

###  Auflisten aller installieten Maps  

In [ ]:
def get_multiverse_maps():
    # return [["WorldName1", "WorldType"], ["WorldName2", "WorldType"], ...]
    value = get_response_command("mvlist")
    value = value.replace("§f", "")
    value = value.replace("§b", "")
    value = value.replace("§d", "")
    value = value.replace("§a", "")
    value = value.replace("§c", "").strip().split("\n")

    maplist = value
    for index, each in enumerate(maplist):
#         print(each.split(" - "))
        maplist[index]=each.split(" - ")
    maplist.pop(0)
    maplist
    return maplist

def get_maps_as_list():
    map_list=get_multiverse_maps()
    map_list
    for index, each in enumerate(map_list):
#         print(each[0])
        map_list[index] = each[0]
    return map_list

In [ ]:
get_multiverse_maps()

In [ ]:
get_maps_as_list()

### Teleportiere den Spieler in eine andere Welt

In [ ]:
target_map = 'world'
# target_map = 'world_the_end'
# target_map = 'world_nether'
# player_name = 
get_response_command("mvtp " + player_name + " " + target_map)

In [ ]:
def teleport_player_to_map(player_name, target_map):
    return get_response_command("mvtp " + player_name + " " + target_map)


In [ ]:
teleport_player_to_map(player_name, target_map)

### Erstelle eine neue Welt
#### 1. eine flache Welt

In [ ]:
new_world_name = "newworld"

In [ ]:
print("mv create " + new_world_name + " " + world_type + " -t FLAT")
get_response_command("mv create newworld normal -t FLAT")

In [ ]:
# teleport to the new map
teleport_player_to_map(player_name, new_world_name)

#### 2. eine Wasserwelt mit ein paar Inseln
- mit dem seed value "gargamel";)

In [ ]:
new_world_name = "MySeedWorld"

In [ ]:

get_response_command('mv create ' + new_world_name + ' NORMAL -s gargamel')

In [ ]:
get_response_command("mvtp " + player_name + " " + new_world_name)

#### weitere Seeds gibt es hier
- https://praxistipps.chip.de/minecraft-die-10-besten-seeds_39316
  - hier das beispiel Nr 8 "***1785852800490497919: Wüste, ein großes Dorf und ein Tempel. Dieser Spawnpunkt ist perfekt für Abenteurer, die sofort mit dem Entdecken loslegen wollen!***"

In [ ]:
new_world_name = "Dorf_und_ein_Tempel"

In [ ]:
get_response_command('mv create ' + new_world_name + ' NORMAL -s 1785852800490497919')

In [ ]:
get_response_command("mvtp " + player_name + " " + new_world_name)

##### zurück zur hauptwelt

In [ ]:
get_response_command("mvtp " + player_name + " " + "world")

## Viel Spass beim Craften